# 1. Positional Encodings:


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('dark_background')
plt.rcParams['axes.prop_cycle'] = plt.cycler(color = sns.color_palette('dark'))
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = (15 , 15)

In [2]:
# def positional_encoding(pos , dmodel):
#     res = []
#     if pos & 1:
#         for i in range(dmodel):
#             res.append(np.cos(pos / 10000 ** (i / dmodel)))
#     else:
#         for i in range(dmodel):
#             res.append(np.sin(pos / 10000 ** (i / dmodel)))

#     return np.array(res)

# #generate positional encodings for 10 positions and 4 dmodel embeddings:
# pencodings = []
# for i in range(20000):
#     pencodings.append(positional_encoding(i , 200))
# pencodings = np.array(pencodings)
# print(pencodings)
# sns.heatmap(pencodings)

In [3]:
#install dependencines:
# !pip3 install datasets transformers sentencepiece

# Sequence to sequence translation using transformers:


In [4]:
from datasets import load_dataset
from evaluate import load
from transformers import pipeline
import transformers
import torch

torch.backends.mps.is_available()

/opt/anaconda3/envs/torch/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
raw_datasets = load_dataset('opus_books' , 'de-en')
raw_datasets['train'][10:18:2]['translation']

[{'de': '»Jane, ich liebe weder Spitzfindigkeiten noch Fragen; außerdem ist es gradezu widerlich, wenn ein Kind ältere Leute in dieser Weise zur Rede stellt.',
  'en': '"Jane, I don\'t like cavillers or questioners; besides, there is something truly forbidding in a child taking up her elders in that manner.'},
 {'de': 'An das Wohnzimmer stieß ein kleines Frühstückszimmer: ich schlüpfte hinein. Hier stand ein großer Bücherschrank.',
  'en': 'A breakfast-room adjoined the drawing-room, I slipped in there.'},
 {'de': 'Ich stieg auf den Sitz in der Fenstervertiefung, zog die Füße nach und kreuzte die Beine wie ein Türke; dann zog ich die dunkelroten Moiree-Vorhänge fest zusammen und saß so in einem doppelten Versteck.',
  'en': 'I mounted into the window- seat: gathering up my feet, I sat cross-legged, like a Turk; and, having drawn the red moreen curtain nearly close, I was shrined in double retirement.'},
 {'de': 'In kurzen Zwischenräumen, wenn ich die Blätter meines Buches wendete, fiel

In [6]:
split_data = raw_datasets['train'].train_test_split(train_size = 0.9 , seed = 20)

#rename test key to validation:
split_data['validation'] = split_data.pop('test')
split_data

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 46320
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 5147
    })
})

In [7]:
model_de_en = 'Helsinki-NLP/opus-mt-de-en'
model_en_de = 'Helsinki-NLP/opus-mt-en-de'
translator = pipeline('translation' , model = model_de_en , framework = 'pt')
translator2 = pipeline('translation' , model = model_en_de , framework = 'pt')

Device set to use mps:0
Device set to use mps:0


In [8]:
print(f"Translating German to English")
i = 4
tdata = split_data['train'][i]['translation']
ttranslation = translator(tdata['de'])
print(f"German sentence: {tdata['de']}")
print(f"Translated sentence: {ttranslation[0]['translation_text']}")
print(f"Original text: {tdata['en']}")

Translating German to English
German sentence: Weil er Heißhunger hatte und jeden Bissen schmatzend kaute, so entstand ein hörbares Geräusch, das den Procurator aufmerksam und erschrocken machte.
Translated sentence: Because he had cravings and chewed every bite, an audible sound arose that made the procurator attentive and terrified.
Original text: As he was very hungry, he made a great deal of noise, and he accented each mouthful strongly, which startled and alarmed the procurator.


In [9]:
print(f"Translating English to German:")
i = 4
tdata = split_data['train']['translation'][i]
ttranslation = translator2(tdata['en'])
print(f"English sentence: {tdata['en']}")
print(f"Translated sentence: {ttranslation[0]['translation_text']}")
print(f"Original text: {tdata['de']}")

Translating English to German:
English sentence: As he was very hungry, he made a great deal of noise, and he accented each mouthful strongly, which startled and alarmed the procurator.
Translated sentence: Da er sehr hungrig war, machte er viel Lärm, und er akzentuierte jeden Mund kräftig, was den Prokurator erschreckte und beunruhigte.
Original text: Weil er Heißhunger hatte und jeden Bissen schmatzend kaute, so entstand ein hörbares Geräusch, das den Procurator aufmerksam und erschrocken machte.


In [10]:
#load tokenizer:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_de_en , return_tensors = 'pt')

In [11]:
#pre-processing a single sample:
i = 4
german_sentence = split_data['train'][i]['translation']['de']
english_sentence = split_data['train'][i]['translation']['en']

inputs = tokenizer(german_sentence)

with tokenizer.as_target_tokenizer():
    targets = tokenizer(english_sentence)

wrong_targets = tokenizer(english_sentence)
print(f"{tokenizer.convert_ids_to_tokens(wrong_targets['input_ids'])}")
print(f"{tokenizer.convert_ids_to_tokens(targets['input_ids'])}")

['▁As', '▁he', '▁was', '▁ver', 'y', '▁', 'hung', 'ry', ',', '▁he', '▁', 'made', '▁a', '▁', 'gre', 'at', '▁de', 'al', '▁of', '▁no', 'ise', ',', '▁and', '▁he', '▁', 'ac', 'cent', 'ed', '▁e', 'ach', '▁', 'mouth', 'ful', '▁str', 'ong', 'ly', ',', '▁wh', 'ich', '▁', 'start', 'led', '▁and', '▁', 'alarm', 'ed', '▁the', '▁pro', 'cur', 'ator', '.', '</s>']
['▁As', '▁he', '▁was', '▁very', '▁hungry', ',', '▁he', '▁made', '▁a', '▁great', '▁deal', '▁of', '▁noise', ',', '▁and', '▁he', '▁accent', 'ed', '▁each', '▁mouth', 'ful', '▁strongly', ',', '▁which', '▁start', 'led', '▁and', '▁alarm', 'ed', '▁the', '▁pro', 'cur', 'ator', '.', '</s>']


/opt/anaconda3/envs/torch/lib/python3.14/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
split_data['train']['translation']

Column([{'de': 'Justin sagte kein Wort.', 'en': 'Justin did not answer.'}, {'de': 'Darauf sprach ein kleiner, dem Aussehen nach noch sehr junger, aber sehr giftiger Herr sich dahin aus, es würde dem Gouvernements-Adelsmarschall jedenfalls angenehm gewesen sein, über die Gelder Rechenschaft abzulegen, und nun gehe er infolge der übertriebenen Feinfühligkeit der Ausschußmitglieder dieser moralischen Befriedigung verlustig.', 'en': 'Then a small, very young-looking, but very venomous man began saying that probably the Marshal of the Province would be pleased to account for the Funds, and that the excessive politeness of the members of the Commission was depriving him of that moral satisfaction.'}, {'de': 'Es ist einem neuen Fürsten niemals zuträglich gewesen, seine Unterthanen zu entwaffnen. Vielmehr hat ein solcher sie allemal mit Nutzen bewaffnet, wenn er sie unbewaffnet fand: denn wenn er sie bewaffnet, so werden diese Waffen Sein, Verdächtige werden treu, die Getreuen können sich erha

In [13]:
max_input_length = 128
max_output_length = 128

def preprocess(dataset):
    #get input and output sequences:
    inputs = [el['de'] for el in dataset['translation']]
    outputs = [el['en'] for el in dataset['translation']]
    minputs = tokenizer(inputs , max_length = max_input_length , truncation = True)

    with tokenizer.as_target_tokenizer():
        targets = tokenizer(outputs , max_length = max_output_length , truncation = True)

    minputs['labels'] = targets['input_ids']
    return minputs

split_data['train'].column_names

['id', 'translation']

In [14]:
tokenized_datasets = split_data.map(
    preprocess, batched = True , remove_columns= split_data['train'].column_names
)

In [15]:
import torch
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM , Seq2SeqTrainingArguments , Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_de_en)
datacollator = DataCollatorForSeq2Seq(tokenizer= tokenizer , model = model , return_tensors = 'pt')

In [16]:
batch = datacollator([tokenized_datasets['train'][i] for i in range(1, 3)])
batch.keys()

KeysView({'input_ids': tensor([[25745,  2817,    53,  5324,     2,    57, 15957,    96,   174,   261,
         16034,     2,   179,   261, 33670,    45,   690,    63, 10249,    74,
             2,    65,   718,    57,  1122,  6533,   776,  5299,     6,    13,
           211,  2243,     6, 20499,  2819, 20999, 13866,  2970,   167,     2,
            81,    11, 21519, 30486,   245, 15177,     2,    10,   953,  8056,
           110, 11255,     9, 36213,    15, 13924,   351, 11926,  2274,     9,
          4808, 20248,   171, 36693, 34412,    17, 12937,   660,     3,     0,
         58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100],
        [  149,    29,   117,   533, 24487,  7528,    24, 22679,  2970,     2,
           363,   890,  8323,    34,    15,    24,  1170,   631,  4860,    34,
            15,     3, 30506,   109,    53,  6897,    76,   212,  1643,    30,
          4597, 31334,     2,   176,   110,    76,  4411,   631,  4860,  1376,
          4567,    16,   632,   176,   110

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="no",      # ⚠️ NEW NAME (important)
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=False,
    logging_steps=1000,
    dataloader_num_workers=0,

    # IMPORTANT for macOS / MPS
    fp16=False,
    bf16=False,
    torch_compile=False
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=datacollator
)
trainer.train()
mdir = './models/transformer_de_en/'
trainer.save_model(mdir)
tokenizer.save_pretrained(mdir)

/var/folders/5f/scjcfk_97_n7zmjltnpm2mjc0000gn/T/ipykernel_5017/4105676410.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/opt/anaconda3/envs/torch/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,2.353900
100,2.130600
150,2.247000
200,2.208200
250,2.282200
300,2.208400
350,2.185200
400,2.237700
450,2.113300
500,2.060900


/opt/anaconda3/envs/torch/lib/python3.14/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/opt/anaconda3/envs/torch/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=23160, training_loss=1.9520133501919235, metrics={'train_runtime': 2601.6754, 'train_samples_per_second': 35.608, 'train_steps_per_second': 8.902, 'total_flos': 1440464407363584.0, 'train_loss': 1.9520133501919235, 'epoch': 2.0})

In [29]:
#load model:
model = AutoModelForSeq2SeqLM.from_pretrained(mdir)
tokenizer = transformers.AutoTokenizer.from_pretrained(mdir)

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


## Inference


In [40]:
text = "Na toll“, sagte er und sah aus dem Fenster."
inputs = tokenizer(text , return_tensors = 'pt' , truncation = True)
outputs = model.generate(**inputs , max_length = 128)
print(tokenizer.decode(outputs[0] , skip_special_tokens = True))

All right, he said, and looked out the window.


In [35]:
inputs

{'input_ids': tensor([[ 103,   29,   53,  261, 8952, 1059,    3,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}